In [ ]:
import math
import numpy as np
from matplotlib.pyplot import *
import seaborn as sns


A = 1 #amplitude
f1 = 1/40. 
f2 = 1/1200.
deltat = 0.8
total_time = int(120 / deltat) #120ms
mini_batch_samples = 15

mini_batch_n = 1

# Threshold voltages for input and output neurons
vthr=1
vthr_neg= -vthr
dlta=[0.3, .5,.8,.9]
v_reset = 0

# Time constants for fast and regular synapses
s_tau=10 #10ms
sf_tau=1 #1ms

# function to add label to graph
def addLabel():
    xlabel('Time (ms)')
    ylabel('Voltage')
    legend()

v_reset = 0

num_inputs = 2
num_outputs = 2

epochs = [50,70,150]

lr = .001
lmbda = 0.001
N = 30 #number of neurons

num_inputs = 2
num_outputs = 2

Nparams= N*N + N*num_inputs + N*num_outputs + N



class AdamOptim():
        def __init__(self, eta=lr,beta1=0.9,beta2=0.999,epsilon=1e-4,Nparams=Nparams):
            # use numpy to vectorize computations
            # replace all m_dw : mean of gradient of W by a np.array

            # dw, db are gradients of weights and biases
            # self.m_dw, self.v_dw = 0, 0
            # self.m_db, self.v_db = 0, 0
            # self.beta1 = beta1
            # self.beta2 = beta2
            # self.epsilon = epsilon
            # self.eta = eta

            self.meanparam = np.zeros((Nparams,),np.float32)
            self.varparam = np.zeros((Nparams,),np.float32)
            self.beta1 = beta1 * np.ones((Nparams,),np.float32)  # beta1 diff for each parameter in case want to make them different
            self.beta2 = beta2 * np.ones((Nparams,),np.float32)
            self.epsilon = epsilon * np.ones((Nparams,),np.float32)
            self.eta = eta * np.ones((Nparams,),np.float32)
            self.multiply1 = np.zeros((Nparams,),np.float32)
            self.multiply2 = np.zeros((Nparams,),np.float32)

        def update(self, t, w, dw):
            ## dw, db are from current minibatch
            ## momentum beta 1
            # # *** weights *** #
            # self.m_dw = self.beta1*self.m_dw + (1-self.beta1)*dw
            # # *** biases *** #
            # self.m_db = self.beta1*self.m_db + (1-self.beta1)*db
            # *** parameters *** #
            np.multiply(self.beta1, self.meanparam, out = self.multiply1)
            # np.multiply(1 - self.beta1, self.meanparam, out = self.multiply2)
            np.multiply(self.beta1, dw, out = self.multiply2)
            # self.multiply2 = dw*(1 - self.beta1)
            # np.multiply(1 - self.beta1, dw, out = self.multiply2)
            self.meanparam = self.multiply1 + self.multiply2

            ## rms beta 2
            # # *** weights *** #
            # self.v_dw = self.beta2*self.v_dw + (1-self.beta2)*(dw**2)
            # # *** biases *** #
            # self.v_db = self.beta2*self.v_db + (1-self.beta2)*(db)
            # *** parameters *** #
            np.multiply(self.beta2, self.varparam, out = self.multiply1)
            np.multiply(1 - self.beta2, np.square(dw), out = self.multiply2)
            self.varparam =  self.multiply1 + self.multiply2

            ## bias correction
            # m_dw_corr = self.m_dw/(1-self.beta1**t)
            # m_db_corr = self.m_db/(1-self.beta1**t)
            # v_dw_corr = self.v_dw/(1-self.beta2**t)
            # v_db_corr = self.v_db/(1-self.beta2**t)
            meanparam_correction = self.meanparam/(1-np.power(self.beta1,t))
            varparam_correction = self.varparam/(1-np.power(self.beta2,t))


            ## update weights and biases
            # w = w - self.eta*(m_dw_corr/(np.sqrt(v_dw_corr)+self.epsilon))
            # b = b - self.eta*(m_db_corr/(np.sqrt(v_db_corr)+self.epsilon))
            w = w - self.eta*(meanparam_correction/(np.sqrt(varparam_correction)+self.epsilon))
            # return w, b
            return w


for delta in dlta:
  for max_epochs in epochs:

    print('Adam as intended, Epochs: ', max_epochs, ' delta ', delta)
    adam = AdamOptim()

    I = np.zeros((num_inputs),np.float32)
    I0 = np.zeros((N),np.float32) #tonic current

    # initialize weights
    Wo = 1.
    W = Wo*np.random.random((N,N)).astype(np.float32)
    Uo = 2.
    U = Uo*np.random.random((N,num_inputs)).astype(np.float32) - (Uo/2)
    v = np.zeros((N),np.float32)
    g = np.zeros((N, total_time),np.float32)
    s = np.zeros((N,total_time),np.float32)

    pv = np.zeros((N,total_time),np.float32) 
    ps = np.zeros((N,total_time),np.float32)
    psdot = np.zeros((N,total_time),np.float32)

    Oo = 0.1
    O = Oo * np.random.random((num_outputs,N)).astype(np.float32) - (Oo/2)
    o = np.zeros((num_inputs,total_time),np.float32)
    od = np.zeros((num_inputs,total_time),np.float32)
    inputs = np.zeros((num_inputs,total_time),np.float32)

    dl_s = np.zeros((N),np.float32)

    plotTime = []
    plotTime.append(time)

    current = np.zeros((N,total_time),np.float32)

    Ohalf = O
    Wf = np.zeros((N,total_time),np.float32)
    UO = np.zeros((N,N),np.float32)
    V = np.zeros((N,total_time),np.float32)
    p0 = -1


    wTime = np.zeros((max_epochs),np.float32)
    uTime = np.zeros((max_epochs),np.float32)
    oTime = np.zeros((max_epochs),np.float32)
    iTime = np.zeros((max_epochs),np.float32)

    for it in range(max_epochs):
        mini_batch_n = 1
        plotTime = []; time = 0; plotTime.append(time)


    ## ZERO OUT GRADIENTS
        dW = np.zeros((N,N),np.float32)
        dO = np.zeros((num_inputs,N),np.float32) 
        dU = np.zeros((N,num_inputs), np.float32)
        dI0 = np.zeros((N),np.float32)
    ## FORWARD FEED    
        for i in range(1,total_time):
            time=time+deltat
            plotTime.append(time)
            
            s1 = 0.2*np.sin(2 * math.pi * f1 * time)
            s2 = 0.8*np.sin(2 * math.pi * f2 * time)
            s3 = 0.3*np.sin(2 * math.pi * f1 * time)
            s4 = 0.7*np.sin(2 * math.pi * f2 * time)
            inputs[:,i] = [A*(s1+s2), A*(s3+s4)]
            
            np.fill_diagonal(W, 0)
            
            # fully connected network structure
            
            I = np.matmul(W,s[:,i]) + A*np.matmul(U,inputs[:,i]) +  I0
            current[:,i] =I[:]      
            
            v = v + (deltat*I)
            
            # gating and threshold
            g[:,i] = np.where(np.logical_or(np.logical_and(v > (vthr - delta),(v <= vthr)), np.logical_and(v >= (-vthr),(v < -vthr + delta))),1./delta,0.) 
            v[v>=vthr] = v_reset
            
            s[:,i] = s[:,i] + deltat*(-s[:,i] + g[:,i]*I)/s_tau
            
            o[:,i] = np.matmul(O,s[:,i])
            #o[:,i] = np.einsum('jk,k->',O, s[:,i])
            #od[:,i] = (-od[:,i-1] + (inputs[:,i]))/sf_tau
            od[:,i] = od[:,i-1]+ deltat * (-od[:,i-1] + inputs[:,i])/sf_tau
            V[:,i] = v
            
            
    ## BACKWARD IN TIME -- PMP            

                  # either at last time stamp of at the last timestamp in a batch
            if (i == total_time - 1) or (i == mini_batch_n * mini_batch_samples):
          
    ## TRAINING THE BATCHES
                for j in range(i, (mini_batch_n - 1)*mini_batch_samples, -1):

                    dl_s= np.matmul(O.T,(o[:,j]-od[:,j]))  +  lmbda*s[:,j]
                    # Tried normalizing the batches
                    # dl_s= (1/(end_col - strt_col)*np.matmul(O.T,(o[:,j]-od[:,j]))  +  lmbda*s[:,j]
                                
                    psdot[:,j-1] = (ps[:,j] + dl_s  -  np.matmul(W.T, pv[:,j]))/s_tau
                    pvdot =  -pv[:,j] + g[:,j]*psdot[:,j-1]
                    
                    ps[:,j] = ps[:,j] + deltat * psdot[:,j-1]
                    pv[:,j] = pv[:,j] + deltat * pvdot 

                #suming up over batches
                strt_col = (mini_batch_n-1)* mini_batch_samples
                end_col = i
                if (i == total_time - 1):
                    end_col= total_time
                
                dW = dW + (np.einsum('ik,jk->ij', pv[:, strt_col:end_col], s[:,strt_col:end_col]) / s_tau)  # .reshape(N,N)
                dO = dO + p0* np.einsum('ik,jk->ij', (o[:, strt_col:end_col]-od[:,strt_col:end_col]), s[:,strt_col:end_col])/ s_tau # reshape(num_input, N)
                dU =dU +(np.einsum('ik,jk->ij', pv[:, strt_col:end_col], inputs[:,strt_col:end_col]) / s_tau)  # .reshape(N, num_input)
                dI0 = dI0 +  (np.einsum('jk->j', pv[:, strt_col:end_col]) / s_tau)  # .reshape(N,1) 
                
                mini_batch_n += 1 

        w_raw = np.concatenate((W.flatten(), U.flatten(), O.flatten(), I0), axis=0)
        dw_raw = np.concatenate((dW.flatten(),dU.flatten(), dO.flatten(), dI0.flatten()), axis=0)
        w_old = w_raw
        w_adam = adam.update(t=it+1, w=w_raw, dw=dw_raw)

        wTime[it] = sum(adam.meanparam[:N*N])
        uTime[it] = sum(adam.meanparam[N*N:N*N + N*num_inputs])
        oTime[it] = sum(adam.meanparam[N*N+N*num_inputs : N*N + N*num_inputs + N*num_outputs])
        iTime[it] = sum(adam.meanparam[N*N + N*num_inputs + N*num_outputs:])


        W = w_adam[: N*N].reshape(N,N)
        U = w_adam[N*N : N*N + N*num_inputs].reshape(N,num_inputs)
        O = p0*w_adam[N*N+N*num_inputs : N*N + N*num_inputs + N*num_outputs].reshape(num_outputs,N)
        I0 = w_adam[N*N + N*num_inputs + N*num_outputs:]

          
          
        if it == max_epochs-1:
          
            figure(1)
            plot(plotTime, inputs[0,:], label = 'input0')
            plot(plotTime,o[0,:], label='output0')
            plot(plotTime,od[0,:], label='desired out 0')
            addLabel()
            show()
            
            figure(2)
            plot(plotTime, inputs[1,:], label = 'input1')
            plot(plotTime,o[1,:], label='output1')
            plot(plotTime,od[1,:], label='desired out 1')
            addLabel()
            show()
            
            figure(3)
            plot(plotTime,V[1,:], label='Voltage')
            plot(plotTime,ps[1,:], label='Ps')
            plot(plotTime,pv[1,:],label='Pv')
            addLabel()
            show()
        
            UO = np.negative(np.matmul(U,O))
            # plot heatmap
            print("UO")
            figure(4)
            m1 = sns.heatmap(UO)
            show()
            
            print("Wf")
            figure(5)
            m3 = sns.heatmap(W)    
            show()
